# Loading Dataset

In [2]:
import pandas as pd
import numpy as np

# Read ingredients and sales data
ingredients = pd.read_excel('../content/Pizza_ingredients.xlsx')
sales = pd.read_excel('../content/Pizza_Sale.xlsx')

In [3]:
ingredients.head()  # Display the first 5 rows of the ingredients data

,pizza_name_id,pizza_name,pizza_ingredients,Items_Qty_In_Grams
0,bbq_ckn_l,The Barbecue Chicken Pizza,Barbecued Chicken,40.0
1,bbq_ckn_l,The Barbecue Chicken Pizza,Red Peppers,15.0
2,bbq_ckn_l,The Barbecue Chicken Pizza,Green Peppers,20.0
3,bbq_ckn_l,The Barbecue Chicken Pizza,Tomatoes,30.0
4,bbq_ckn_l,The Barbecue Chicken Pizza,Red Onions,60.0


In [4]:
sales.head()    # Display the first 5 rows of the sales data

,pizza_id,order_id,pizza_name_id,quantity,order_date,order_time,unit_price,total_price,pizza_size,pizza_category,pizza_ingredients,pizza_name
0,1,1,hawaiian_m,1,2015-01-01 00:00:00,11:38:36,13.25,13.25,M,Classic,"Sliced Ham, Pineapple, Mozzarella Cheese",The Hawaiian Pizza
1,2,2,classic_dlx_m,1,2015-01-01 00:00:00,11:57:40,16.00,16.00,M,Classic,"Pepperoni, Mushrooms, Red Onions, Red Peppers,...",The Classic Deluxe Pizza
2,3,2,five_cheese_l,1,2015-01-01 00:00:00,11:57:40,18.50,18.50,L,Veggie,"Mozzarella Cheese, Provolone Cheese, Smoked Go...",The Five Cheese Pizza
3,4,2,ital_supr_l,1,2015-01-01 00:00:00,11:57:40,20.75,20.75,L,Supreme,"Calabrese Salami, Capocollo, Tomatoes, Red Oni...",The Italian Supreme Pizza
4,5,2,mexicana_m,1,2015-01-01 00:00:00,11:57:40,16.00,16.00,M,Veggie,"Tomatoes, Red Peppers, Jalapeno Peppers, Red O...",The Mexicana Pizza


# Changing Datatypes

In [5]:
ingredients.dtypes

,0
pizza_name_id,object
pizza_name,object
pizza_ingredients,object
Items_Qty_In_Grams,float64


In [6]:
ingredients['pizza_name_id'] = ingredients['pizza_name_id'].astype('category')
ingredients['pizza_name'] = ingredients['pizza_name'].astype('category')
ingredients['pizza_ingredients'] = ingredients['pizza_ingredients'].astype('category')

In [7]:
ingredients.dtypes

,0
pizza_name_id,category
pizza_name,category
pizza_ingredients,category
Items_Qty_In_Grams,float64


In [8]:
sales.dtypes

,0
pizza_id,int64
order_id,int64
pizza_name_id,object
quantity,int64
order_date,object
order_time,object
unit_price,float64
total_price,float64
pizza_size,object
pizza_category,object


In [9]:
sales['pizza_id'] = sales['pizza_id'].astype('category')
sales['order_id'] = sales['order_id'].astype('category')
sales['pizza_name_id'] = sales['pizza_name_id'].astype('category')
sales['order_date'] = pd.to_datetime(sales['order_date'], errors='coerce')    # Convert 'order_date' to datetime
sales['order_time'] = pd.to_datetime(sales['order_time'], format='%H:%M:%S')  # Convert 'order_time' to datetime with specified format
sales['pizza_size'] = sales['pizza_size'].astype('category')
sales['pizza_category'] = sales['pizza_category'].astype('category')
sales['pizza_ingredients'] = sales['pizza_ingredients'].astype('category')
sales['pizza_name'] = sales['pizza_name'].astype('category')

In [10]:
sales.dtypes

,0
pizza_id,category
order_id,category
pizza_name_id,category
quantity,int64
order_date,datetime64[ns]
order_time,datetime64[ns]
unit_price,float64
total_price,float64
pizza_size,category
pizza_category,category


# Handling Missing Values

**Drop Duplicate Values**

In [11]:
sales.drop_duplicates(inplace=True)
ingredients.drop_duplicates(inplace=True)

**Filling missing values for ingredients table**

In [12]:
ingredients.isna().sum()

,0
pizza_name_id,0
pizza_name,0
pizza_ingredients,0
Items_Qty_In_Grams,4


In [13]:
ingredients['Items_Qty_In_Grams'] = ingredients['Items_Qty_In_Grams'].interpolate(method='linear')

**Filling missing values for sales table**

In [14]:
sales.isna().sum()

,0
pizza_id,0
order_id,0
pizza_name_id,16
quantity,0
order_date,0
order_time,0
unit_price,0
total_price,7
pizza_size,0
pizza_category,23


In [15]:
# Filling Missing Pizza Name Based on Pizza Ingredients
ingredients_name_mapping = sales[['pizza_ingredients', 'pizza_name']].dropna().drop_duplicates()
ingredients_name_mapping = ingredients_name_mapping.set_index('pizza_ingredients')['pizza_name'].to_dict()
sales['pizza_name'] = sales['pizza_name'].fillna(sales['pizza_ingredients'].map(ingredients_name_mapping))

# Filling Total Price Based on Quantity
sales['total_price'] = sales['total_price'].fillna(sales['unit_price'] * sales['quantity'])

# Filling Missing Pizza Categories Based on Pizza Name ID
category_mapping = sales[['pizza_name_id', 'pizza_category']].dropna().drop_duplicates()
category_mapping = category_mapping.set_index('pizza_name_id')['pizza_category'].to_dict()
sales['pizza_category'] = sales['pizza_category'].fillna(sales['pizza_name_id'].map(category_mapping))

# Filling Missing Pizza Ingredients Based on Pizza Name
ingredients_mapping = sales[['pizza_name', 'pizza_ingredients']].dropna().drop_duplicates()
ingredients_mapping = ingredients_mapping.set_index('pizza_name')['pizza_ingredients'].to_dict()
sales['pizza_ingredients'] = sales['pizza_ingredients'].fillna(sales['pizza_name'].map(ingredients_mapping))

# Filling Missing Pizza Name ID Based on Pizza Name
name_id_mapping = sales[['pizza_name', 'pizza_name_id']].dropna().drop_duplicates(subset='pizza_name', keep='first')
name_id_mapping = name_id_mapping.set_index('pizza_name')['pizza_name_id'].to_dict()
sales['pizza_name_id'] = sales['pizza_name_id'].astype('object')
sales['pizza_name_id'] = sales['pizza_name_id'].fillna(sales['pizza_name'].map(name_id_mapping))
sales['pizza_name_id'] = sales['pizza_name_id'].astype('category')

In [16]:
sales.isna().sum()

,0
pizza_id,0
order_id,0
pizza_name_id,0
quantity,0
order_date,0
order_time,0
unit_price,0
total_price,0
pizza_size,0
pizza_category,0


# Feature Engineering

In [17]:
sales = sales.sort_values('order_date')   # Sort the DataFrame by 'order_date'
sales.rename(columns={'order_date': 'date'}, inplace=True)  # Rename 'order_date' to 'date'

# Extracting date and time components
sales['Year'] = sales['date'].dt.year # Extract year from 'date'
sales['Month'] = sales['date'].dt.month # Extract month from 'date'
sales['Day'] = sales['date'].dt.day # Extract day from 'date'
sales['Hour'] = sales['order_time'].dt.hour # Extract hour from 'order_time'
sales['Minute'] = sales['order_time'].dt.minute # Extract minute from 'order_time'
sales['Second'] = sales['order_time'].dt.second # Extract second from 'order_time'

sales.set_index('date', inplace=True)
sales.drop('order_time', axis=1, inplace=True) # Drop 'order_time' column

In [ ]:
sales.head()

,pizza_id,order_id,pizza_name_id,quantity,unit_price,total_price,pizza_size,pizza_category,pizza_ingredients,pizza_name,Year,Month,Day,Hour,Minute,Second
date,,,,,,,,,,,,,,,,
2015-01-01,1,1,hawaiian_m,1,13.25,13.25,M,Classic,"Sliced Ham, Pineapple, Mozzarella Cheese",The Hawaiian Pizza,2015,1,1,11,38,36
2015-01-01,104,42,peppr_salami_m,1,16.50,16.50,M,Supreme,"Genoa Salami, Capocollo, Pepperoni, Tomatoes, ...",The Pepper Salami Pizza,2015,1,1,17,28,9
2015-01-01,105,43,ckn_alfredo_m,1,16.75,16.75,M,Chicken,"Chicken, Red Onions, Red Peppers, Mushrooms, A...",The Chicken Alfredo Pizza,2015,1,1,17,38,34
2015-01-01,106,43,ital_supr_m,1,16.50,16.50,M,Supreme,"Calabrese Salami, Capocollo, Tomatoes, Red Oni...",The Italian Supreme Pizza,2015,1,1,17,38,34
2015-01-01,107,43,peppr_salami_l,1,20.75,20.75,L,Supreme,"Genoa Salami, Capocollo, Pepperoni, Tomatoes, ...",The Pepper Salami Pizza,2015,1,1,17,38,34


# Removing Outliers

In [18]:
from scipy.stats import zscore  # Importing the zscore function
import numpy as np  # Importing the numpy library

# Function to remove outliers
def remove_outliers(df, threshold=3):
    # Select only numerical columns
    numerical_df = df.select_dtypes(include=['float', 'int64'])
    z_scores = np.abs((numerical_df - numerical_df.mean()) / numerical_df.std()) # Calculate Z-scores for numerical columns

    '''Create a mask for rows where all numerical Z-scores are below the threshold
    and then apply the mask to the original DataFrame to keep all columns'''

    mask = (z_scores < threshold).all(axis=1)
    df_clean = df[mask].copy()
    return df_clean  # Returning the cleaned DataFrame

# Assuming `ingredients` and `sales` are pre-defined pandas DataFrames
print("Before removing outliers of ingredients:", sales.shape)
df = remove_outliers(sales)  # Remove outliers for `ingredients`
print("After removing outliers of ingredients:", sales.shape)

Before removing outliers of ingredients: (48620, 16)
After removing outliers of ingredients: (48620, 16)


# Save the processed data

In [19]:
sales.to_csv('processed_sales.csv')
ingredients.to_csv('processed_ingredients.csv')